In [5]:
import requests
import json
import pandas as pd
from datetime import datetime

webpage = requests.get("https://wusa.ca/events/")
jsonscript =str(webpage.content)
isolatedinformation=jsonscript.split('<script type="application/ld+json">')[1].split("</script>")[0][4:-4].encode("utf16", errors="surrogatepass").decode("utf16").encode().decode('unicode_escape')

json = json.loads(isolatedinformation.replace('&lt;p&gt;',"").replace("[&hellip;]&lt;/p&gt;\\\\n",""))

postsDf ={}
postscolumns = ['account','date','caption',"display_photo",'event_details']
postsDf = pd.DataFrame(columns = postscolumns)

In [6]:
import time

time_now = int(time.time())
for event in json:
    if int(datetime.fromisoformat(event["startDate"]).timestamp()) >= time_now:
        try: 
            location = event["location"]["address"]["streetAddress"]
        except Exception as err:
            print(str(err))
            location = None
        
        new_row = pd.DataFrame({
                "account": ["WUSA"],
                "date": [time_now],
                "caption": str(event["description"])+" [For More Information, Click View Post] ",
                "display_photo": event["image"],
                "url": [event['url']],
                "likes": [0],
                "event_details": [{
                    "is_event": True,
                    "event_name": event["name"],
                    "event_description": str(event["description"])+" ... ",
                    "categories": ["Social"],
                    "start_time": int(datetime.fromisoformat(event["startDate"]).timestamp()+ 7200000),
                    "end_time": int(datetime.fromisoformat(event["endDate"]).timestamp()+ 7200000),
                    "location": location,
                }]
        })

        postsDf = pd.concat([postsDf, new_row])

print(postsDf)
postsDf.to_csv(path_or_buf="WUSA events")

'location'
'streetAddress'
'streetAddress'
'streetAddress'
  account        date                                            caption  \
0    WUSA  1725330671  📢 Hey WUSA Members! If this is your first time...   
0    WUSA  1725330671  CIBC x WUSA x UWaterloo presents Dillon Franci...   
0    WUSA  1725330671  Welcome Week 🍂 Get ready to FALL back into fun...   
0    WUSA  1725330671  Rise and shine, Warriors! WUSA’s inviting you ...   
0    WUSA  1725330671  Esports Gaming Lounge Game on, Warriors! Brace...   
0    WUSA  1725330671  😍 A Welcome Week favourite is back! Are you re...   
0    WUSA  1725330671  Step right up! The WUSA Carnival is back for T...   
0    WUSA  1725330671  To give you a warm welcome back to campus, we ...   
0    WUSA  1725330671  Step right up! The WUSA Carnival is back for T...   
0    WUSA  1725330671  Don\'t miss out on the ultimate bowling experi...   
0    WUSA  1725330671  Esports Gaming Lounge Game on, Warriors! Brace...   
0    WUSA  1725330671  The Ca

In [7]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

# Remove all documents with these IDs
result = collection.delete_many({'account': "WUSA"})

# Print the number of documents deleted
print(f"Documents removed: {result.deleted_count}")


Documents removed: 21


In [8]:
from typing import List
from together import Together
import time

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

insertObjectIds = []

for index, row in postsDf.iterrows():
    info = f'"id": "{int(index)}"|* "account": "{row["account"]}"|* "date": "{row["date"]}"|* "caption": "{row["caption"]}"|*'
    embbededtext = ',\n'.join(x for x in info.replace('\n','\\n').split('|*')) 

    row_dict = {}
    for column in postsDf.columns.tolist():
        row_dict[column] = row[column]
    row_dict["embedded"] = generate_embedding([embbededtext], embedding_model_string)  
    result = collection.insert_one(row_dict)
    print(f"Inserted document ID: {result.inserted_id}")
    insertObjectIds.append(result.inserted_id)
    time.sleep(0.5)


Inserted document ID: 66d674f679ba601bff9a5694
Inserted document ID: 66d674f779ba601bff9a5695
Inserted document ID: 66d674f879ba601bff9a5696
Inserted document ID: 66d674fa79ba601bff9a5697
Inserted document ID: 66d674fb79ba601bff9a5698
Inserted document ID: 66d674fc79ba601bff9a5699
Inserted document ID: 66d674fd79ba601bff9a569a
Inserted document ID: 66d674ff79ba601bff9a569b
Inserted document ID: 66d6750079ba601bff9a569c
Inserted document ID: 66d6750179ba601bff9a569d
Inserted document ID: 66d6750379ba601bff9a569e
Inserted document ID: 66d6750479ba601bff9a569f
Inserted document ID: 66d6750679ba601bff9a56a0
Inserted document ID: 66d6750779ba601bff9a56a1
Inserted document ID: 66d6750979ba601bff9a56a2
Inserted document ID: 66d6750a79ba601bff9a56a3
Inserted document ID: 66d6750c79ba601bff9a56a4
Inserted document ID: 66d6750d79ba601bff9a56a5
Inserted document ID: 66d6750e79ba601bff9a56a6
Inserted document ID: 66d6751079ba601bff9a56a7
Inserted document ID: 66d6751179ba601bff9a56a8
